In [ ]:
#| default_exp core

# <span style="color:DarkOrange"> Core </span>

HierarchicalForecast contains pure Python implementations of hierarchical reconciliation methods as well as a `core.HierarchicalReconciliation` wrapper class that enables easy interaction with these methods through pandas DataFrames containing the hierarchical time series and the base predictions.

The `core.HierarchicalReconciliation` reconciliation class operates with the hierarchical time series pd.DataFrame `Y_df`, the base predictions pd.DataFrame `Y_hat_df`, the aggregation constraints matrix `S`. For more information on the creation of aggregation constraints matrix see the utils [aggregation method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br><br>

In [ ]:
#| export
import re
from inspect import signature
from scipy.stats import norm
from typing import Callable, Dict, List, Optional

import numpy as np
import pandas as pd

from hierarchicalforecast.probabilistic_methods import Normality, Bootstrap, PERMBU

In [ ]:
#| hide
from fastcore.test import test_close, test_eq, test_fail
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#| exporti
def _build_fn_name(fn) -> str:
    fn_name = type(fn).__name__
    func_params = fn.__dict__

    # Take default parameter out of names
    args_to_remove = ['insample']
    if not func_params.get('nonnegative', False):
        args_to_remove += ['nonnegative']

    if fn_name == 'MinTrace' and \
        func_params['method']=='mint_shrink':
        if func_params['mint_shr_ridge'] == 2e-8:
            args_to_remove += ['mint_shr_ridge']

    func_params = [f'{name}-{value}' for name, value in func_params.items() if name not in args_to_remove]
    if func_params:
        fn_name += '_' + '_'.join(func_params)
    return fn_name

In [ ]:
#| hide
# test fn name
from hierarchicalforecast.methods import BottomUp, MinTrace

In [ ]:
#| hide
test_eq(_build_fn_name(BottomUp()), 'BottomUp')
test_eq(
    _build_fn_name(MinTrace(method='ols')), 
    'MinTrace_method-ols'
)
test_eq(
    _build_fn_name(MinTrace(method='ols', nonnegative=True)), 
    'MinTrace_method-ols_nonnegative-True'
)
test_eq(
    _build_fn_name(MinTrace(method='mint_shr')), 
    'MinTrace_method-mint_shr'
)

# <span style="color:DarkBlue"> core.HierarchicalReconciliation </span>

In [ ]:
#| exporti
def _reverse_engineer_sigmah(Y_hat_df, y_hat, model_name, uids):
    """
    This function assumes that the model creates prediction intervals
    under a normality assumption with the following the Equation:
    $\hat{y}_{t+h} + c \hat{sigma}_{h}$

    In the future, we might deprecate this function in favor of a 
    direct usage of an estimated $\hat{sigma}_{h}$
    """
    drop_cols = ['ds', 'y'] if 'y' in Y_hat_df.columns else ['ds']
    model_names = Y_hat_df.drop(columns=drop_cols, axis=1).columns.to_list()
    pi_model_names = [name for name in model_names if ('-lo' in name or '-hi' in name)]
    pi_model_name = [pi_name for pi_name in pi_model_names if model_name in pi_name]
    pi = len(pi_model_name) > 0

    if not pi:
        raise Exception(f'Please include {model_name} prediction intervals in `Y_hat_df`')

    pi_col = pi_model_name[0]
    sign = -1 if 'lo' in pi_col else 1
    level_col = re.findall('[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', pi_col)
    level_col = float(level_col[0])
    z = norm.ppf(0.5 + level_col / 200)
    sigmah = Y_hat_df.pivot(columns='ds', values=pi_col).loc[uids].values
    sigmah = sign * (sigmah - y_hat) / z

    return sigmah

In [ ]:
#| export
class HierarchicalReconciliation:
    """Hierarchical Reconciliation Class.

    The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
    HierarchicaForecast methods for a collection of time series and base predictions stored in 
    pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
    y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
    example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

    **Parameters:**<br>
    `reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

    **References:**<br>
    [Rob J. Hyndman and George Athanasopoulos (2018). \"Forecasting principles and practice, Hierarchical and Grouped Series\".](https://otexts.com/fpp3/hierarchical.html)
    """
    def __init__(self,
                 reconcilers: List[Callable]):
        self.reconcilers = reconcilers
        self.insample = any([method.insample for method in reconcilers])

    def reconcile(self, 
                  Y_hat_df: pd.DataFrame,
                  S: pd.DataFrame,
                  tags: Dict[str, np.ndarray],
                  Y_df: Optional[pd.DataFrame] = None,
                  level: Optional[List[int]] = None,
                  intervals_method: str = 'normality'):
        """Hierarchical Reconciliation Method.

        The `reconcile` method is analogous to SKLearn `fit` method, it applies different 
        reconciliation methods instantiated in the `reconcilers` list. 

        Most reconciliation methods can be described by the following convenient 
        linear algebra notation:

        $$\\tilde{\mathbf{y}}_{[a,b],\\tau} = \mathbf{S}_{[a,b][b]} \mathbf{P}_{[b][a,b]} \hat{\mathbf{y}}_{[a,b],\\tau}$$

        where $a, b$ represent the aggregate and bottom levels, $\mathbf{S}_{[a,b][b]}$ contains
        the hierarchical aggregation constraints, and $\mathbf{P}_{[b][a,b]}$ varies across 
        reconciliation methods. The reconciled predictions are $\\tilde{\mathbf{y}}_{[a,b],\\tau}$, and the 
        base predictions $\hat{\mathbf{y}}_{[a,b],\\tau}$.

        **Parameters:**<br>
        `Y_hat_df`: pd.DataFrame, base forecasts with columns `ds` and models to reconcile indexed by `unique_id`.<br>
        `Y_df`: pd.DataFrame, training set of base time series with columns `['ds', 'y']` indexed by `unique_id`.
        If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
        `S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
        `tags`: Each key is a level and its value contains tags associated to that level.<br>
        `level`: float list 0-100, confidence levels for prediction intervals.<br>
        `intervals_method`: str, method used to calculate prediction intervals, one of `normality`, `bootstrap`, `permbu`.<br>

        **Returns:**<br>
        `y_tilde`: pd.DataFrame, with reconciled predictions.        
        """
        #----------------------------- Preliminary Wrangling/Protections -----------------------------#
        # Check input's validity
        if intervals_method not in ['normality', 'bootstrap', 'permbu']:
            raise ValueError(f'Unkwon interval method: {intervals_method}')

        if self.insample or (intervals_method in ['bootstrap', 'permbu']):
            if Y_df is None:
                raise Exception('you need to pass `Y_df`')            

        # Declare output names
        drop_cols = ['ds', 'y'] if 'y' in Y_hat_df.columns else ['ds']
        model_names = Y_hat_df.drop(columns=drop_cols, axis=1).columns.to_list()
        pi_model_names = [name for name in model_names if ('-lo' in name or '-hi' in name)]
        model_names = [name for name in model_names if name not in pi_model_names]

        uids = Y_hat_df.index.unique()

        # Check Y_hat_df\S_df series difference
        S_diff  = len(S.index.difference(uids))
        Y_hat_diff = len(Y_hat_df.index.difference(S.index.unique()))
        if S_diff > 0 or Y_hat_diff > 0:
            raise Exception(f'Check `S_df`, `Y_hat_df` series difference, S\Y_hat={S_diff}, Y_hat\S={Y_hat_diff}')

        if Y_df is not None:
            # Check Y_hat_df\Y_df series difference
            Y_diff  = len(Y_df.index.difference(uids))
            Y_hat_diff = len(Y_hat_df.index.difference(Y_df.index.unique()))
            if Y_diff > 0 or  Y_hat_diff > 0:
                raise Exception(f'Check `Y_hat_df`, `Y_df` series difference, Y_hat\Y={Y_hat_diff}, Y\Y_hat={Y_diff}')

        # Same Y_hat_df/S_df/Y_df's unique_id order to prevent errors
        S_ = S.loc[uids]
    

        #---------------------------------------- Predictions ----------------------------------------#
        # Initialize reconciler arguments
        reconciler_args = dict(
            S=S_.values.astype(np.float32),
            idx_bottom=S_.index.get_indexer(S.columns),
            tags={key: S_.index.get_indexer(val) for key, val in tags.items()}
        )
        if Y_df is not None:
            reconciler_args['y_insample'] = Y_df.pivot(columns='ds', values='y').loc[uids].values.astype(np.float32)

        fcsts = Y_hat_df.copy()
        for reconcile_fn in self.reconcilers:
            reconcile_fn_name = _build_fn_name(reconcile_fn)
            has_fitted = 'y_hat_insample' in signature(reconcile_fn).parameters
            has_level = 'level' in signature(reconcile_fn).parameters
            
            # TODO: maybe sort in advance by uids and avoid .loc[uids]
            # This change affects y_hat_model, y_insample, y_hat_insample, sigmah
            # change pivot for df.values and reshapes.
            for model_name in model_names:
                y_hat = Y_hat_df.pivot(columns='ds', values=model_name).loc[uids].values
                reconciler_args['y_hat'] = y_hat

                if (self.insample and has_fitted) or intervals_method in ['bootstrap', 'permbu']:
                    y_hat_insample = Y_df.pivot(columns='ds', values=model_name).loc[uids].values
                    y_hat_insample = y_hat_insample.astype(np.float32)
                    reconciler_args['y_hat_insample'] = y_hat_insample

                if has_level and (level is not None):
                    reconciler_args['level'] = level

                    if intervals_method in ['normality', 'permbu']:
                        sigmah = _reverse_engineer_sigmah(Y_hat_df=Y_hat_df,
                                    y_hat=y_hat, model_name=model_name, uids=uids)

                    reconciler_args['intervals_method'] = intervals_method

                # Mean reconciliation
                kwargs = [key for key in signature(reconcile_fn).parameters if key in reconciler_args.keys()]
                kwargs = {key: reconciler_args[key] for key in kwargs}
                fcsts_model = reconcile_fn(**kwargs)

                # TODO: instantiate prob reconcilers after mean reconc 
                # and use _prob_reconcile function from probabilistic_methods.py
                # this will greatly simplify code above, and improve its readability
                # this will require outputs of reconcile_fn to include P, W

                fcsts[f'{model_name}/{reconcile_fn_name}'] = fcsts_model['mean'].flatten()
                if intervals_method in ['bootstrap', 'normality', 'permbu'] and level is not None:
                    for lv in level:
                        fcsts[f'{model_name}/{reconcile_fn_name}-lo-{lv}'] = fcsts_model[f'lo-{lv}'].flatten()
                        fcsts[f'{model_name}/{reconcile_fn_name}-hi-{lv}'] = fcsts_model[f'hi-{lv}'].flatten()
                    del reconciler_args['level']
                    del reconciler_args['sampler']
                if self.insample and has_fitted:
                    del reconciler_args['y_hat_insample']
        return fcsts

In [ ]:
show_doc(HierarchicalReconciliation, 
         name='init', title_level=3)

In [ ]:
show_doc(HierarchicalReconciliation)

In [ ]:
show_doc(HierarchicalReconciliation.reconcile,
         name='reconcile', title_level=3)

In [ ]:
#| hide
from hierarchicalforecast.methods import (
    BottomUp, TopDown, MiddleOut, MinTrace, ERM,
)
from hierarchicalforecast.utils import aggregate

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# non strictly hierarchical structure
hierS_grouped_df = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]
# strictly hierarchical structure
hiers_strictly = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'State', 'Region'], 
]

# getting df
hier_grouped_df, S_grouped_df, tags_grouped = aggregate(df, hierS_grouped_df)
hier_strict_df, S_strict, tags_strict = aggregate(df, hiers_strictly)

In [ ]:
#| hide
hier_grouped_df['y_model'] = hier_grouped_df['y']
# we should be able to recover y using the methods
hier_grouped_hat_df = hier_grouped_df.groupby('unique_id').tail(12)
ds_h = hier_grouped_hat_df['ds'].unique()
hier_grouped_df = hier_grouped_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_grouped_df['y_model'] += np.random.uniform(-1, 1, len(hier_grouped_df))

#hierachical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df, Y_df=hier_grouped_df, 
                            S=S_grouped_df, tags=tags_grouped)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-5
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# test expected error
# different series S and Y_hat_df
test_fail(
    hrec.reconcile,
    contains='series difference',
    args=(hier_grouped_hat_df.drop('Australia'), S_grouped_df, tags_grouped, hier_grouped_df),
    
)
test_fail(
    hrec.reconcile,
    contains='series difference',
    args=(hier_grouped_hat_df, S_grouped_df.drop('Australia'), tags_grouped, hier_grouped_df),
    
)
test_fail(
    hrec.reconcile,
    contains='series difference',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df.drop('Australia')),
    
)

In [ ]:
#| hide
# test reconcile method without insample
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df,
                            S=S_grouped_df, tags=tags_grouped)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-5
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# top down should break
# with non strictly hierarchical structures
hrec = HierarchicalReconciliation([TopDown(method='average_proportions')])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped,  hier_grouped_df,)
)

In [ ]:
#| hide
# methods should work with
# srtictly hierarchical structures
#| hide
hier_strict_df['y_model'] = hier_strict_df['y']
# we should be able to recover y using the methods
hier_strict_df_h = hier_strict_df.groupby('unique_id').tail(12)
ds_h = hier_strict_df_h['ds'].unique()
hier_strict_df = hier_strict_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_strict_df['y_model'] += np.random.uniform(-1, 1, len(hier_strict_df))

middle_out_level = 'Country/State'
# hierarchical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
    # top down doesnt recover the original y
    # but it should recover the total level
    TopDown(method='forecast_proportions'),
    TopDown(method='average_proportions'),
    TopDown(method='proportion_averages'),
    # middle out doesnt recover the original y
    # but it should recover the total level
    MiddleOut(middle_level=middle_out_level, top_down_method='forecast_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='average_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='proportion_averages'),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(
    Y_hat_df=hier_strict_df_h, 
    Y_df=hier_strict_df, 
    S=S_strict, 
    tags=tags_strict
)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-5
    if 'TopDown' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict['Country']
        test_close(reconciled['y'].loc[total_tag], 
                   reconciled[model].loc[total_tag], 1e-2)
    elif 'MiddleOut' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict[middle_out_level]
        test_close(reconciled['y'].loc[total_tag], 
                   reconciled[model].loc[total_tag], 1e-2)
    else:
        test_close(reconciled['y'], reconciled[model], eps)

In [ ]:
#| hide
# MinTrace should break
# with extremely overfitted model, y_model==y

zero_df = hier_grouped_df.copy()
zero_df['y'] = 0
zero_df['y_model'] = 0
hrec = HierarchicalReconciliation([MinTrace(method='mint_shrink')])
test_fail(
    hrec.reconcile,
    contains='Insample residuals',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped,  zero_df)
)

In [ ]:
#| hide
#test methods that dont use residuals
#even if their signature includes
#that argument
hrec = HierarchicalReconciliation([MinTrace(method='ols')])
reconciled = hrec.reconcile(
    Y_hat_df=hier_grouped_hat_df, 
    Y_df=hier_grouped_df.drop(columns=['y_model']), 
    S=S_grouped_df, 
    tags=tags_grouped
)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    test_close(reconciled['y'], reconciled[model])

In [ ]:
#| hide
reconciled.loc[tags_grouped['Country/State']]

In [ ]:
#| hide
#test methods bootstrap prediction
#intervals
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(hier_grouped_hat_df, 
                            Y_df=hier_grouped_df, S=S_grouped_df, tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='bootstrap')
total = reconciled.loc[tags_grouped['Country/State/Region/Purpose']].groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.loc['Australia'][['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods that require prediction intervals
# normality
hier_grouped_hat_df['y_model-lo-80'] = hier_grouped_hat_df['y_model'] - 1.96
hier_grouped_hat_df['y_model-hi-80'] = hier_grouped_hat_df['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(hier_grouped_hat_df, 
                            Y_df=hier_grouped_df, S=S_grouped_df, tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='normality')
total = reconciled.loc[tags_grouped['Country/State/Region/Purpose']].groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.loc['Australia'][['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods that require prediction intervals
# PERMBU

# test expect error with grouped structure 
# (non strictly hierarchical)
hier_grouped_hat_df['y_model-lo-80'] = hier_grouped_hat_df['y_model'] - 1.96
hier_grouped_hat_df['y_model-hi-80'] = hier_grouped_hat_df['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df, [80, 90], 'permbu',)
)

# test PERMBU
hier_strict_df_h['y_model-lo-80'] = hier_strict_df_h['y_model'] - 1.96
hier_strict_df_h['y_model-hi-80'] = hier_strict_df_h['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(hier_strict_df_h, 
                            Y_df=hier_strict_df, S=S_strict, tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='permbu')
total = reconciled.loc[tags_grouped['Country/State/Region']].groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.loc['Australia'][['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

# <span style="color:DarkBlue"> Example </span>

In [ ]:
#| eval: false
import numpy as np
import pandas as pd

from statsforecast.core import StatsForecast
from statsforecast.models import ETS, Naive

from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace

# Load TourismSmall dataset
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# Create hierarchical seires based on geographic levels and purpose
# And Convert quarterly ds string to pd.datetime format
hierarchy_levels = [['Country'],
                    ['Country', 'State'], 
                    ['Country', 'Purpose'], 
                    ['Country', 'State', 'Region'], 
                    ['Country', 'State', 'Purpose'], 
                    ['Country', 'State', 'Region', 'Purpose']]

Y_df, S_df, tags = aggregate(df=df, spec=hierarchy_levels)
qs = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()
Y_df = Y_df.reset_index()

# Split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(4)
Y_train_df = Y_df.drop(Y_test_df.index)

# Compute base auto-ETS predictions
# Careful identifying correct data freq, this data quarterly 'Q'
fcst = StatsForecast(df=Y_train_df,
                     #models=[ETS(season_length=12), Naive()],
                     models=[Naive()],
                     freq='Q', n_jobs=-1)
Y_hat_df = fcst.forecast(h=4, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

# Reconcile the base predictions
Y_train_df = Y_train_df.reset_index().set_index('unique_id')
Y_hat_df = Y_hat_df.reset_index().set_index('unique_id')
reconcilers = [BottomUp(),
               MinTrace(method='mint_shrink')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)
Y_rec_df.groupby('unique_id').head(2)